**Задание**

Для каждой страны необходимо подсчитать количество дней, когда количество умерших было больше, чем в среднем по миру за все время наблюдений. Ответом к задаче будет топ-10 стран с наибольшим показателем количества таких дней.

Важные моменты
* Считаются дни, когда было строго больше среднего
* Разные регионы страны вначале необходимо совместить, чтобы статистика считалась верно (смотри последний запрос выше)
* Доработка данных уже после запроса на Python вполне возможна, если это требуется
* Не обязательно делать ровно один агрегационный запрос - можно сделать серию запросов. Главное, чтобы по итогу был подсчитан запрашиваемый набор данных

Результатом должен быть файл `result.json` в следующем формате:
```
{
    "<country 1>": "<number of days>",
    "<country 2>": "<number of days>",
    ...
    "<country 10>": "<number of days>"
}
```

In [1]:
from pymongo import MongoClient
import json
client = MongoClient('localhost', 27017)
world = client.covid.world  # коллекция world внутри базы covid

# Функция, чтобы красиво печатать результаты:

def p(x):
    from pprint import pprint
    pprint(list(x))

In [2]:
avg_deaths = list(world.aggregate([
    {
        '$group': {
            "_id": {
                "country": "$country",
                "date": "$date"
            },
            "deaths": {'$sum': "$deaths"},
        }
    },
    {
        '$group': {
            "_id": None,
            "avg_deaths": {"$avg": "$deaths"}
        }
    }
]))[0]['avg_deaths']

avg_deaths

1027.9561092447432

In [3]:
res = list(world.aggregate([
    {
        '$group': {
            "_id": {
                "country": "$country",
                "date": "$date"
            },
            "deaths": {'$sum': "$deaths"},
        }
    },
    {
        "$group": {
            "_id": {
                "country": "$_id.country",
                "date": "$_id.date"
            },
            "avg_deaths": {"$avg": "$deaths"}
        }
    },
    {
        "$match": {
            "avg_deaths": {"$gt": avg_deaths}
        }
    },
    {
        "$group": {
            "_id": {
                "country": "$_id.country",
            },
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {'count': -1},
    },
    {
        "$limit": 10
    }
]))

p(res)

[{'_id': {'country': 'China'}, 'count': 151},
 {'_id': {'country': 'Italy'}, 'count': 120},
 {'_id': {'country': 'Iran'}, 'count': 115},
 {'_id': {'country': 'Spain'}, 'count': 113},
 {'_id': {'country': 'France'}, 'count': 109},
 {'_id': {'country': 'US'}, 'count': 108},
 {'_id': {'country': 'United Kingdom'}, 'count': 106},
 {'_id': {'country': 'Netherlands'}, 'count': 102},
 {'_id': {'country': 'Germany'}, 'count': 100},
 {'_id': {'country': 'Belgium'}, 'count': 99}]


In [4]:
answer = {}
for i in res:
     answer[i['_id']['country']] = i['count']

In [5]:
with open("result.json", 'w') as f:
    f.write(
        json.dumps(
            answer
        )
    )

In [6]:
! cat result.json

{"China": 151, "Italy": 120, "Iran": 115, "Spain": 113, "France": 109, "US": 108, "United Kingdom": 106, "Netherlands": 102, "Germany": 100, "Belgium": 99}

Более подробную информацию про составление агрегационных пайплайнов в MongoDB можно найти в [официальной документации](https://docs.mongodb.com/manual/core/aggregation-pipeline/).